In [ ]:
import os
import math
import random
from PIL import Image
import matplotlib.pyplot as plt
import zipfile



In [ ]:

if not os.path.exists('./data/train'):
    with zipfile.ZipFile('./train.zip') as train_zip:
        train_zip.extractall('./data')
    with zipfile.ZipFile('./test.zip') as test_zip:
        test_zip.extractall('./data')

train_dir = './data/train'
test_dir = './data/test'



In [ ]:
def get_filelist_and_labels(folder):
    file_list = []
    label_list = []
    for fname in os.listdir(folder):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            file_list.append(os.path.join(folder, fname))
            # Tên file: cat.0.jpg hay dog.123.jpg...
            prefix = fname.split('.')[0]
            label = 1 if prefix == 'dog' else 0
            label_list.append(label)
    return file_list, label_list




In [ ]:
X_train_files, y_train = get_filelist_and_labels(train_dir)
X_test_files, y_test = get_filelist_and_labels(test_dir)

MAX_TRAIN = 1000
MAX_TEST = 200
X_train_files, y_train = X_train_files[:MAX_TRAIN], y_train[:MAX_TRAIN]
X_test_files, y_test = X_test_files[:MAX_TEST], y_test[:MAX_TEST]




In [ ]:
def load_images(filelist, img_size=(32,32)):
    X = []
    for fname in filelist:
        img = Image.open(fname).convert('L').resize(img_size)
        arr = [[img.getpixel((j, i))/255.0 for j in range(img_size[1])] for i in range(img_size[0])]
        X.append([arr]) # [1][H][W]
    return X



In [ ]:
X_train = load_images(X_train_files)
X_test = load_images(X_test_files)

# Shuffle train set
train_data = list(zip(X_train, y_train))
random.shuffle(train_data)
X_train, y_train = zip(*train_data)




In [ ]:
class Conv2D:
    def __init__(self, in_size, kernel_size, filters):
        self.kernel_size = kernel_size
        self.filters = filters
        self.kernels = [[[[
            random.uniform(-0.1, 0.1) for _ in range(kernel_size)
        ] for _ in range(kernel_size)] for _ in range(1)] for _ in range(filters)]
        self.bias = [random.uniform(-0.05, 0.05) for _ in range(filters)]
    def forward(self, x):
        h = len(x[0])
        w = len(x[0][0])
        out_size = h - self.kernel_size + 1
        out = [[[0 for _ in range(out_size)] for _ in range(out_size)] for _ in range(self.filters)]
        for f in range(self.filters):
            for i in range(out_size):
                for j in range(out_size):
                    s = 0
                    for ki in range(self.kernel_size):
                        for kj in range(self.kernel_size):
                            s += x[0][i+ki][j+kj] * self.kernels[f][0][ki][kj]
                    out[f][i][j] = s + self.bias[f]
        return out



In [ ]:
class MaxPool2D:
    def __init__(self, pool_size):
        self.pool_size = pool_size
    def forward(self, x):
        C = len(x)
        H = len(x[0])
        W = len(x[0][0])
        out_H = H // self.pool_size
        out_W = W // self.pool_size
        out = [[[0 for _ in range(out_W)] for _ in range(out_H)] for _ in range(C)]
        for c in range(C):
            for i in range(out_H):
                for j in range(out_W):
                    mx = -math.inf
                    for pi in range(self.pool_size):
                        for pj in range(self.pool_size):
                            mx = max(mx, x[c][i*self.pool_size+pi][j*self.pool_size+pj])
                    out[c][i][j] = mx
        return out



In [ ]:
class Flatten:
    def forward(self, x):
        return [v for c in x for row in c for v in row]



In [ ]:
class Dense:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = [[random.uniform(-0.1, 0.1) for _ in range(in_features)] for _ in range(out_features)]
        self.bias = [random.uniform(-0.05, 0.05) for _ in range(out_features)]
        self.last_input = None
    def forward(self, x):
        self.last_input = x
        return [sum(self.weights[i][j]*x[j] for j in range(self.in_features)) + self.bias[i] for i in range(self.out_features)]
    def backward(self, grad_out, lr):
        for i in range(self.out_features):
            for j in range(self.in_features):
                self.weights[i][j] -= lr * grad_out[i] * self.last_input[j]
            self.bias[i] -= lr * grad_out[i]



In [ ]:
def relu(x):
    if isinstance(x[0], list):
        return [relu(sub) for sub in x]
    return [max(0, v) for v in x]

def softmax(x):
    m = max(x)
    exps = [math.exp(i-m) for i in x]
    sum_exps = sum(exps)
    return [e/sum_exps for e in exps]

def cross_entropy(pred, label):
    eps = 1e-10
    return -math.log(pred[label] + eps)

def cross_entropy_grad(pred, label):
    grad = [p for p in pred]
    grad[label] -= 1
    return grad

def get_flatten_size(img_size):
    size = img_size
    size = (size - 3 + 1) // 2
    return size * size * 8



In [ ]:

img_size = 32
epochs = 5
lr = 0.001


In [ ]:

conv1 = Conv2D(img_size, 3, 8)
pool1 = MaxPool2D(2)
flatten = Flatten()
flatten_size = get_flatten_size(img_size)
dense1 = Dense(flatten_size, 2)




In [ ]:
batch_size = 10
for ep in range(epochs):
    total_loss = 0
    for bi in range(0, len(X_train), batch_size):
        grad_sum = [0]*2
        for idx in range(bi, min(bi+batch_size, len(X_train))):
            xi, yi = X_train[idx], y_train[idx]
            x = conv1.forward(xi)
            x = relu(x)
            x = pool1.forward(x)
            x = flatten.forward(x)
            logits = dense1.forward(x)
            probs = softmax(logits)
            loss = cross_entropy(probs, yi)
            total_loss += loss
            grad_logits = cross_entropy_grad(probs, yi)
            # Cộng dồn grad
            grad_sum = [grad_sum[i]+grad_logits[i] for i in range(2)]
        # Update dense1
        dense1.backward([g/batch_size for g in grad_sum], lr)
    print(f"Epoch {ep+1}, Loss: {total_loss/len(X_train):.4f}")



In [ ]:

plt.figure(figsize=(10,5))
for i in range(8):
    x = conv1.forward(X_test[i])
    x = relu(x)
    x = pool1.forward(x)
    x = flatten.forward(x)
    logits = dense1.forward(x)
    probs = softmax(logits)
    pred = probs.index(max(probs))
    plt.subplot(2,4,i+1)
    plt.imshow(X_test[i][0], cmap='gray')
    plt.title("Pred: %s" % ("dog" if pred==1 else "cat"))
    plt.axis('off')
plt.tight_layout()
plt.show()